# 5Andar

## Abordagem exploratória
### Por iuli hardt

##

In [1]:
#import libs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#As abas dos arquivos foram separadas por questões de facilidade de uso em LLM e reaproveitadas aqui
media_custo = pd.read_csv(r'C:\Users\iulih\Meu Drive\Work\PS\QuintoAndar\data_csv\media_plan.csv',sep=',')
leads_gerados = pd.read_csv(r'C:\Users\iulih\Meu Drive\Work\PS\QuintoAndar\data_csv\leads.csv',sep=',')
operacao = pd.read_csv(r'C:\Users\iulih\Meu Drive\Work\PS\QuintoAndar\data_csv\operantions.csv',sep=',')

Correções e transformações de STR em Float:

In [5]:
operacao['ticket_medio'] = operacao['ticket_medio'].replace('[\$,]', '', regex=True).astype(float)
operacao['tenant_prospect_to_contract_signed'] = operacao['tenant_prospect_to_contract_signed'].replace('[\%,]', '', regex=True).astype(float)

In [6]:
media_custo['daily_cost'] = media_custo['daily_cost'].replace('[\$,]', '', regex=True).astype(float)


Agrupamento:

In [33]:
# 1. Agrupar dados
tabela_combinada = pd.merge(leads_gerados,media_custo, left_on=['dt_event', 'city_group'], right_on=['dt_insertion_order', 'city_group'])

# 2. Calcular métricas
metricas = tabela_combinada.groupby(['city_group']).agg({'daily_cost': 'sum', 'qualifieds': 'sum'}).reset_index()

In [41]:
metricas_ops = pd.merge(metricas,operacao, on=['city_group'])

metricas_ops[metricas_ops['city_group']=='Florianopolis']#.sample(3)

,city_group,daily_cost,qualifieds,nbr_bairros_abertos,nbr_bairros_sem_ops,leads,leads_5a,leads_nops,contratos_assinados,tenant_prospects,tenant_prospect_to_contract_signed,ticket_medio
4,Florianopolis,360500.06,3310.0,59,76,165986,155370,10616,771,20304,3.8,2554.0


In [49]:
metricas_ops['Retorno_esperado'] = metricas_ops['qualifieds'] * metricas_ops['tenant_prospect_to_contract_signed'] / 100 * metricas_ops['ticket_medio'] 

metricas_ops['Total_contratos_estimado'] = metricas_ops['qualifieds'] * metricas_ops['tenant_prospect_to_contract_signed'] / 100

metricas_ops['ROI'] = ((metricas_ops['Retorno_esperado'] - metricas_ops['daily_cost']) / metricas_ops['daily_cost']) * 100

In [52]:
metricas_ops_final = metricas_ops[['city_group','daily_cost','qualifieds','ticket_medio','tenant_prospect_to_contract_signed','Total_contratos_estimado','Retorno_esperado','ROI']]

In [53]:
metricas_ops_final.sort_values(by='ROI', ascending=False)

,city_group,daily_cost,qualifieds,ticket_medio,tenant_prospect_to_contract_signed,Total_contratos_estimado,Retorno_esperado,ROI
8,Sao Paulo,5939676.80,62156.0,3671.0,13.81,8583.7436,3.151092e+07,430.515781
0,Belo Horizonte,560335.83,8285.0,2623.0,8.38,694.2830,1.821104e+06,225.002295
7,Rio de Janeiro,4489481.55,29039.0,3499.0,10.65,3092.6535,1.082119e+07,141.034393
2,Campinas,455075.78,2691.0,3024.0,12.01,323.1891,9.773238e+05,114.760680
4,Florianopolis,360500.06,3310.0,2554.0,3.80,125.7800,3.212421e+05,-10.889857
1,Brasilia,608111.59,3718.0,1714.0,7.99,297.0682,5.091749e+05,-16.269497
5,Goiania,435077.92,2916.0,1941.0,6.11,178.1676,3.458233e+05,-20.514626
6,Porto Alegre,1866590.25,5320.0,2155.0,11.87,631.4840,1.360848e+06,-27.094443
3,Curitiba,738854.97,3778.0,1914.0,7.09,267.8602,5.126844e+05,-30.610953


In [56]:
metricas_ops_final['CAC'] =metricas_ops_final['daily_cost'] /metricas_ops_final['qualifieds'] 

C:\Users\iulih\AppData\Local\Temp\ipykernel_23328\272738879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metricas_ops_final['CAC'] =metricas_ops_final['daily_cost'] /metricas_ops_final['qualifieds']


### Resultado final:

In [58]:
metricas_ops_final.sort_values(by='CAC', ascending=False)

,city_group,daily_cost,qualifieds,ticket_medio,tenant_prospect_to_contract_signed,Total_contratos_estimado,Retorno_esperado,ROI,CAC
6,Porto Alegre,1866590.25,5320.0,2155.0,11.87,631.4840,1.360848e+06,-27.094443,350.862829
3,Curitiba,738854.97,3778.0,1914.0,7.09,267.8602,5.126844e+05,-30.610953,195.567753
2,Campinas,455075.78,2691.0,3024.0,12.01,323.1891,9.773238e+05,114.760680,169.110286
1,Brasilia,608111.59,3718.0,1714.0,7.99,297.0682,5.091749e+05,-16.269497,163.558792
7,Rio de Janeiro,4489481.55,29039.0,3499.0,10.65,3092.6535,1.082119e+07,141.034393,154.601796
5,Goiania,435077.92,2916.0,1941.0,6.11,178.1676,3.458233e+05,-20.514626,149.203676
4,Florianopolis,360500.06,3310.0,2554.0,3.80,125.7800,3.212421e+05,-10.889857,108.912405
8,Sao Paulo,5939676.80,62156.0,3671.0,13.81,8583.7436,3.151092e+07,430.515781,95.560795
0,Belo Horizonte,560335.83,8285.0,2623.0,8.38,694.2830,1.821104e+06,225.002295,67.632568


----

## Regressão para identificar o custo por lead de cada Canal de comunicação

In [79]:
import pandas as pd
import statsmodels.api as sm

merged_df = pd.merge(leads_gerados,media_custo, left_on=['dt_event', 'city_group'], right_on=['dt_insertion_order', 'city_group'], how='inner')

In [80]:
# Criar variáveis dummy para o tipo de mídia
merged_df = pd.get_dummies(merged_df, columns=['media'], prefix='media')

Transformando em algebra linear:

In [81]:
merged_df.sample(3)

,dt_event,city_group,leads_5a,discards,qualifieds,first_listing,dt_insertion_order,daily_cost,media_JORNAL,media_OUT OF HOME,media_RÁDIO,media_TV ABERTA
614,2021-04-29,Porto Alegre,922,265,86,47,2021-04-29,3020.44,False,False,True,False
815,2021-05-13,Sao Paulo,11616,3,850,725,2021-05-13,5104.52,False,False,True,False
325,2021-04-02,Rio de Janeiro,4742,863,97,190,2021-04-02,6527.11,False,True,False,False


In [82]:
# Definir as variáveis independentes (X) e a variável dependente (y)
X = merged_df[['daily_cost', 'media_RÁDIO', 'media_TV ABERTA','media_JORNAL', 'media_OUT OF HOME']]
y = merged_df['qualifieds']


In [84]:
results = sm.OLS(y,X*1).fit() #X*1 tranforma True e False em 1 e 0

Resultado final os parametros desejados:

In [85]:
results.params

daily_cost             0.003306
media_RÁDIO          273.817210
media_TV ABERTA      192.264561
media_JORNAL         147.705206
media_OUT OF HOME    234.889117
dtype: float64

Avaliação de cada variavel para identificar o P valor:

In [86]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             qualifieds   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     28.22
Date:                Wed, 27 Dec 2023   Prob (F-statistic):           3.36e-22
Time:                        10:08:06   Log-Likelihood:                -6929.6
No. Observations:                 976   AIC:                         1.387e+04
Df Residuals:                     971   BIC:                         1.389e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
daily_cost            0.0033      0.000     10.502      0.000       0.003       0.004
media_RÁDIO         273.8172     17.927     15.274      0.000     238.637     308.997
media_TV ABERTA     192.2646     16.464     11.678      0.000     159.956     224.573
media_JORNAL        147.7052    120.186      1.229      0.219     -88.149     383.559
media_OUT OF HOME   234.8891     17.421     13.483      0.000     200.702     269.077
==============================================================================
Omnibus:                      213.646   Durbin-Watson:                   0.672
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              369.232
Skew:                           1.400   Prob(JB):                     6.64e-81
Kurtosis:                       4.111   Cond. No.                     4.38e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.38e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

media_JORNAL esta estatisticamente invalido pois o P valor é maior que 0,05, sendo igual a 0,219. Para os outras são validadas.